In [1]:
import json
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import tensorflow as tf
from pandas import read_csv
from numpy import array
from numpy import hstack
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from pandas import read_csv
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat

import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import chart_studio.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)

import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

# 전처리

In [2]:
# 데이터 폴더 경로
path = './data/'

In [3]:
table = pd.DataFrame()

arrs = os.listdir(path)

In [4]:
fnames = []
for arr in arrs:
    fnames.append(arr)

In [5]:
# 데이터 가져오기
for i in range(len(fnames)): 
    data = pd.read_excel(path+fnames[i], skiprows=2)
    table = table.append(data)

In [6]:
rawdata = table.copy()

In [7]:
rawdata

,번호,측정소명,년/월,회차,수심(m),수소이온농도,용존산소(㎎/L),BOD(㎎/L),COD(㎎/L),부유물질(㎎/L),...,벤젠(㎎/L),폴리크로리네이티트비페닐(㎎/L),유기인(㎎/L),안티몬 (㎎/L),클로로포름 (㎎/L),디에틸헥실프탈레이트 (㎎/L),1.4-다이옥세인 (㎎/L),투명도(m),유량(㎥/s),포름알데히드(㎎/L)
0,1,잠실,2017/01/11,1회차,0.5,8.5,14.3,1.5,4.1,2.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,잠실,2017/02/15,1회차,0.5,8.5,15.9,1.9,4.0,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,잠실,2017/03/15,1회차,0.5,7.4,13.7,1.4,4.3,4.8,...,0,NaN,NaN,0,0,NaN,0,NaN,NaN,0
3,4,잠실,2017/04/12,1회차,0.5,8.2,12.2,1.2,3.7,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,잠실,2017/05/17,1회차,0.5,8.1,12.6,0.5,3.7,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,8,잠실,2014/08/13,1회차,0.5,7.6,8.6,1.2,3.7,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,잠실,2014/09/17,1회차,0.5,7.9,9.8,1.5,3.6,6.6,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9,10,잠실,2014/10/15,1회차,0.5,8.1,9.9,1.3,4.5,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,잠실,2014/11/12,1회차,0.5,7.7,11.5,0.6,3.3,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
rawdata = rawdata[['년/월', '용존산소(㎎/L)', 'BOD(㎎/L)', 'COD(㎎/L)', '부유물질(㎎/L)' ]]

In [9]:
rawdata.rename(columns = {'년/월':'date', '용존산소(㎎/L)':'DO', 'BOD(㎎/L)':'BOD', 'COD(㎎/L)':'COD', '부유물질(㎎/L)':'SS'}, inplace = True)

In [10]:
rawdata

,date,DO,BOD,COD,SS
0,2017/01/11,14.3,1.5,4.1,2.4
1,2017/02/15,15.9,1.9,4.0,3.4
2,2017/03/15,13.7,1.4,4.3,4.8
3,2017/04/12,12.2,1.2,3.7,3.6
4,2017/05/17,12.6,0.5,3.7,1.2
...,...,...,...,...,...
7,2014/08/13,8.6,1.2,3.7,5.8
8,2014/09/17,9.8,1.5,3.6,6.6
9,2014/10/15,9.9,1.3,4.5,8.0
10,2014/11/12,11.5,0.6,3.3,3.6


In [11]:
rawdata.reset_index(inplace=True)
del rawdata['index']

In [12]:
type(rawdata['date'][0])

str

In [13]:
dates= []

In [14]:
for i in range(len(rawdata)):
    dates.append(str(rawdata['date'][i])[:-3])

In [15]:
rawdata['dates'] = dates

In [16]:
rawdata

,date,DO,BOD,COD,SS,dates
0,2017/01/11,14.3,1.5,4.1,2.4,2017/01
1,2017/02/15,15.9,1.9,4.0,3.4,2017/02
2,2017/03/15,13.7,1.4,4.3,4.8,2017/03
3,2017/04/12,12.2,1.2,3.7,3.6,2017/04
4,2017/05/17,12.6,0.5,3.7,1.2,2017/05
...,...,...,...,...,...,...
294,2014/08/13,8.6,1.2,3.7,5.8,2014/08
295,2014/09/17,9.8,1.5,3.6,6.6,2014/09
296,2014/10/15,9.9,1.3,4.5,8.0,2014/10
297,2014/11/12,11.5,0.6,3.3,3.6,2014/11


In [17]:
year = []
mon = []

In [18]:
for i in range(len(rawdata)):
    year.append(str(rawdata['dates'][i])[:4])

In [19]:
for i in range(len(rawdata)):
    mon.append(str(rawdata['dates'][i])[5:])

In [20]:
rawdata['year'] = year
rawdata['month'] = mon

In [21]:
rawdata

,date,DO,BOD,COD,SS,dates,year,month
0,2017/01/11,14.3,1.5,4.1,2.4,2017/01,2017,01
1,2017/02/15,15.9,1.9,4.0,3.4,2017/02,2017,02
2,2017/03/15,13.7,1.4,4.3,4.8,2017/03,2017,03
3,2017/04/12,12.2,1.2,3.7,3.6,2017/04,2017,04
4,2017/05/17,12.6,0.5,3.7,1.2,2017/05,2017,05
...,...,...,...,...,...,...,...,...
294,2014/08/13,8.6,1.2,3.7,5.8,2014/08,2014,08
295,2014/09/17,9.8,1.5,3.6,6.6,2014/09,2014,09
296,2014/10/15,9.9,1.3,4.5,8.0,2014/10,2014,10
297,2014/11/12,11.5,0.6,3.3,3.6,2014/11,2014,11


In [22]:
df = rawdata.copy()

In [23]:
df = df[['year', 'month','dates', 'DO', 'BOD', 'COD','SS']]

# 월별

In [24]:
mon = pd.DataFrame(df['DO'].groupby(df['month']).mean())
mon

,DO
month,
,NaN
01,13.176000
02,14.241667
03,13.192000
04,11.691667
05,10.272000
06,9.448000
07,8.783333
08,8.892000


In [25]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14},
    'xaxis': {
    'title': {
      'text': '월'}
    },
    'yaxis': {
    'title': {
      'text': '용존 산소량'}
    }
}
layout['title'] = "송파구 잠실동 한강 월별 용존 산소량 그래프"

fig = mon.iplot(kind='line', color='blue',
               layout=layout, asFigure=True)
fig

In [26]:
fig.write_image("/home/datastory/no_18.(신규)한강의 용존 산소량을 통한 수질 예측/18/img_1_1.png")
                
fig_json = fig.to_json()
fig_json
json_obj = json.loads(fig_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./18/img_1_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 년도별

In [27]:
years = pd.DataFrame(df['DO'].groupby(df['year']).mean())
years

,DO
year,
,NaN
1996,10.033333
1997,10.591667
1998,11.800000
1999,8.566667
2000,8.608333
2001,9.558333
2002,9.366667
2003,9.941667


In [28]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14},
    'xaxis': {
    'title': {
      'text': '연도'}
    },
    'yaxis': {
    'title': {
      'text': '용존 산소량'}
    }
}
layout['title'] = "송파구 잠실동 한강 연도별 용존 산소량 그래프"

fig = years.iplot(kind='line', color='blue',
                 layout=layout, asFigure=True)
fig

In [29]:
fig.write_image("/home/datastory/no_18.(신규)한강의 용존 산소량을 통한 수질 예측/18/img_2_1.png")
                
fig_json = fig.to_json()
fig_json
json_obj = json.loads(fig_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./18/img_2_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 상관분석

In [30]:
import seaborn as sns    
import matplotlib as mpl
import matplotlib.font_manager as fm

In [31]:
total = df[['DO', 'BOD', 'COD','SS']]

In [32]:
# import seaborn as sns    
# plt.figure(figsize=(10,10))
# font_path='C:\Windows\\Fonts\\malgunbd.ttf'
# font_name = fm.FontProperties(fname=font_path).get_name()
# plt.rc('font', family=font_name)
# sns.heatmap(data = total.corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [33]:
corr = total.corr()

z = corr.values
x = corr.columns
y = corr.index

annotations = []
for n,row in enumerate(z):
    for m, val in enumerate(row):
        annotations.append(
            dict(text='{0:.2f}'.format(z[n][m]) if not np.isnan(z[n][m]) else '',
                x=x[m],
                y=y[n],
                xref='x1',
                yref='y1',
                showarrow=False,
                font=dict(
                    family="Noto Sans KR",
                    size=14,
                    color="#ffffff")))
        
trace = go.Heatmap(x=x, y=y, z=z)
fig = go.Figure(data=trace)

fig.update_layout(
    annotations=annotations,
    #title_text="7일 데이터",
    #title_x=0.5,
    font=dict(
        family="Noto Sans KR",
        size=14
    )
)

fig.show()

In [34]:
fig.write_image("/home/datastory/no_18.(신규)한강의 용존 산소량을 통한 수질 예측/18/img_3_1.png")
                
fig_json = fig.to_json()
fig_json
json_obj = json.loads(fig_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./18/img_3_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)

# 계절별

In [35]:
sea = df['DO'].groupby(df['month']).sum()

In [36]:
se = ['겨울','봄','여름','가을']

In [37]:
sea = pd.DataFrame(sea)

In [38]:
sea.reset_index(inplace=True)

In [39]:
sea

,month,DO
0,,0.0
1,01,329.4
2,02,341.8
3,03,329.8
4,04,280.6
5,05,256.8
6,06,236.2
7,07,210.8
8,08,222.3
9,09,219.7


In [40]:
win = (sea['DO'][1] + sea['DO'][2] + sea['DO'][12]) / 3
spr = (sea['DO'][3] + sea['DO'][4] + sea['DO'][5]) / 3
summ = (sea['DO'][6] + sea['DO'][7] + sea['DO'][8]) / 3 
aut = (sea['DO'][9] + sea['DO'][10]+ sea['DO'][11]) / 3 

In [41]:
season_list = []
season_list.append(win)
season_list.append(spr)
season_list.append(summ)
season_list.append(aut)

In [42]:
tab = pd.DataFrame(season_list)
tab.index = se

In [43]:
layout = {
    'font': {
        'family': 'Noto Sans KR',
        'size': 14},
    'xaxis': {
    'title': {
      'text': '계절'}
    },
    'yaxis': {
    'title': {
      'text': '용존 산소량'}
    }
}
layout['title'] = "송파구 잠실동 한강 계절별 용존 산소량 그래프"

fig = tab.iplot(kind='bar', color='powderblue',
                layout=layout, asFigure=True)
fig

In [44]:
fig.write_image("/home/datastory/no_18.(신규)한강의 용존 산소량을 통한 수질 예측/18/img_4_1.png")
                
fig_json = fig.to_json()
fig_json
json_obj = json.loads(fig_json, encoding="utf-8")
json_obj_d = json.dumps(json_obj, ensure_ascii=False)

json_obj_d
                
with open('./18/img_4_1.json', 'w', encoding='utf-8') as make_file:
    json.dump(json_obj, make_file, indent="\t", ensure_ascii=False)